AI Agent with Pydantic to get the Approximate price of the Product in $$

Create a simple assistant that uses any LLM and it should use pydantic

When we ask about any product it should give you two 
information of product Name, product details tentative price in USD (integer).

Use chat Prompt Template.

In [ ]:
#import libraries
import os
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from dataclasses import dataclass
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser

In [3]:
#create pydantic product class
@dataclass
class Product(BaseModel):
    product_name: str = Field(description="The name of the product")
    product_description: str = Field(description="The description of the product")
    product_price: float = Field(description="The price of the product in USD")
    

In [7]:
#evnironment variables
load_dotenv()
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")
os.environ["LANGCHAIN_TRACEING_V2"]= "true"

In [31]:
llm = ChatGroq(model="llama-3.1-8b-instant")
print(llm)


client=<groq.resources.chat.completions.Completions object at 0x115309a70> async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x11530a3f0> model_name='llama-3.1-8b-instant' model_kwargs={} groq_api_key=SecretStr('**********')


In [69]:
parser = PydanticOutputParser(pydantic_object=Product)

prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a product information specialist. Your response must be a valid JSON object with the following structure:
    {{
        "product_name": "name of product",
        "product_description": "brief description",
        "product_price": "Price of the product in USD"
    }}"""),
    ("human", "Please provide product information for {query}\n\n{format_instructions}")
])

In [79]:
#create a chain
chain = prompt| llm | parser

response = chain.invoke({
    "query": "Apple iphone 16 pro max 128GB",
    "format_instructions": parser.get_format_instructions()
})

In [82]:
#print(f"Product Name is:{response.product_name}")
#print(f"Product Description is: {response.product_description}")
#print(f"Product Price is: ${response.product_price:.2f}")

for key, value in response.model_dump().items():
    print(f"{key}: {value}")

product_name: Apple iPhone 16 Pro Max
product_description: The latest iPhone model with a 6.7-inch display, up to 16GB of RAM, and a quad-camera setup.
product_price: 1599.99
